In [1]:
import pandas as pd

In [8]:
data = pd.read_csv('adult_300.csv')
data = data.drop(columns=['capital-gain', 'capital-loss'])
# data.to_csv("adult_150.csv", index=False) # save the dataset without anonymity(for later comparison of model performance)
k = 4
quasi_identifier = ['age', 'education-num', 'hours-per-week']
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,54,Private,1st-4th,2,Married-civ-spouse,Other-service,Husband,White,Male,40,Mexico,<=50K
1,24,Private,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,<=50K
2,20,Private,Some-college,10,Never-married,Machine-op-inspct,Own-child,White,Female,40,United-States,<=50K
3,38,Self-emp-not-inc,Masters,14,Married-civ-spouse,Farming-fishing,Husband,White,Male,70,United-States,<=50K
4,37,Private,Bachelors,13,Never-married,Tech-support,Not-in-family,White,Female,40,United-States,<=50K


In [9]:
def mondrian(partition):
    ranks = {}

    for qi in quasi_identifier:
        ranks[qi] = len(set(partition[qi]))
    
    ranks = sorted(ranks.items(), key=lambda t: t[1], reverse=True)
    print(ranks)

    return anonymize(partition, ranks)

def anonymize(partition, ranks):
    dim = ranks[0][0]

    partition = partition.sort_values(by=dim)
    si = partition[dim].count()
    mid = si//2

    lhs = partition[:mid]
    rhs = partition[mid:]

    if(len(lhs)>=k and len(rhs)>=k):
        return pd.concat([anonymize(lhs, ranks), anonymize(rhs, ranks)])
    return summarized(partition, dim)

def summarized(partition, dim):
    for qi in quasi_identifier:
        partition = partition.sort_values(by=qi)
        s = f"[{partition[qi].iloc[0]}-{partition[qi].iloc[-1]}]"
        partition[qi] = [s] * partition[qi].size
        
    
    return partition

In [10]:
result = mondrian(data)
print(result)
result.to_csv(f'adult_with_anonymity_k{k}.csv', index=False)

[('age', 54), ('hours-per-week', 41), ('education-num', 15)]
         age         workclass     education education-num  \
56   [17-18]           Private          10th         [6-7]   
136  [17-18]           Private          11th         [6-7]   
61   [17-18]           Private          11th         [6-7]   
32   [17-18]      Self-emp-inc          10th         [6-7]   
117  [19-20]           Private       HS-grad        [9-10]   
..       ...               ...           ...           ...   
104  [67-90]           Private       HS-grad        [4-14]   
7    [67-90]  Self-emp-not-inc       7th-8th        [4-14]   
137  [67-90]           Private       HS-grad        [4-14]   
112  [67-90]           Private       Masters        [4-14]   
273  [67-90]  Self-emp-not-inc  Some-college        [4-14]   

         marital-status         occupation   relationship   race     sex  \
56        Never-married      Other-service      Own-child  White    Male   
136       Never-married              Sales